<p align="center">
    <img width=210 height=150 src="../reports/figures/cunef_bw.png">
</p>

<div>
<h1>2.0 Data Preprocessing </h1>
Fake News Classifier <br>
<strong>Ciencia de Datos para la Información No Estructurada</strong> <br>
<strong>Master Universidatio en Ciencia de Datos</strong>
</div>

<div style='text-align:right'>Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>

---

En este notebook nos centramos en elaborar el modelo que utilizaremos para detectar las noticias verdaderas y falsas.

In [22]:
# Libraries
import pandas as pd
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carviagu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

### Lectura de datos

In [3]:
# Train data
X_train = pd.read_csv("../data/interim/train.csv")
Y_train = X_train['label']
X_train = X_train['title']

# Test data
X_test = pd.read_csv("../data/interim/test.csv")
Y_test = X_test['label']
X_test = X_test['title']

### Procesado de los datos
Realizaremos la limpieza de los datos, seguiremos el mismo procedimiento que en el análisis realizado. Crearemos una función que realizará la limpieza de los datos para cada fila del dataset, además incorporaremos la lematización con el objetivo de reducir la variabilidad de las palabras, verbos especialmente que permitirán manejar un menor vocabulario al modelo. 

In [8]:
# Translator to clean data 
translator = str.maketrans('', '', string.punctuation)

# Word lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [17]:
def title_cleaner(tlt):
    """
    Cleans the text passed. All text to lower, blanck spaces and punctuation.
    Also eliminates stopwords and lematize verbs
    :param tlt: text to clean
    :type tlt: string
    """
    # Lower letters and blank spaces out, we eliminate all type of punctuation
    # Finally we split the data (tokenize)
    tokens = re.sub(r"[^a-zA-Z0-9]", " ",tlt.strip().lower().translate(translator)).split()

    # Lemmatize and eliminate any kind of stopword that doesn't gives us any information
    final_words = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in tokens if not word in stopwords]

    # Joining the resulted words cleaned
    return " ".join(final_words)

Aplicamos la función para limpiar los datos (tanto en train como en test)

In [18]:
X_train = X_train.map(lambda x : title_cleaner(x))
X_test = X_test.map(lambda x : title_cleaner(x))

Ahora codificamos la variable objetivo mediante LabelEncoding...

In [21]:
lb = preprocessing.LabelEncoder()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.transform(Y_test)

print(lb.classes_)

['FAKE' 'REAL']


### Creación de variables del modelo
Ahora procedemos a crear las variables con los datos que serán utilizadas para el clasificador de noticias. Utilizaremos la técnica del Bag of Words

In [29]:
# Model for creating variables
corpus = X_train
cv = CountVectorizer()

# Creating the train result 
cv_matrix = cv.fit_transform(corpus)

In [35]:
# Preparing train data
features = cv.get_feature_names_out()
data = cv_matrix.toarray()
X_train = pd.DataFrame(data, columns=features)

In [37]:
# Preparing test data
features = cv.get_feature_names_out()
data = cv.transform(X_test).toarray()
X_test = pd.DataFrame(data, columns=features)

### Guardado de los datos
Una vez procesados guardamos los datos

In [38]:
X_train['label'] = Y_train
X_train.to_csv("../data/processed/train.csv")

X_test['label'] = Y_test
X_test.to_csv("../data/processed/test.csv")

---

<div style='text-align:center'> Elaborado por Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>